In [2]:
import pandas as pd

c:\Users\cxh83\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\cxh83\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [3]:
df = pd.read_csv("building_data/hospital.csv")
df

,POI_YM,POI_ID,POI_LON,POI_LAT,CATEGORY,POI_X,POI_Y
0,201901,46519,121.535296,24.982699,居家護理,304040.848781,2.763968e+06
1,201901,79499,121.531718,25.101441,長照服務,303627.957961,2.777119e+06
2,201901,79573,121.523161,25.123221,長照服務,302755.552467,2.779528e+06
3,201901,79526,121.458382,25.041406,長照服務,296253.877467,2.770442e+06
4,201901,81250,121.545210,24.922685,長照服務,305068.422927,2.757324e+06
...,...,...,...,...,...,...,...
3219,202401,49891,121.563023,25.026578,區域醫院,306819.898722,2.768839e+06
3220,202401,96882,121.577649,25.043017,長照服務,308288.207880,2.770667e+06
3221,202401,81480,121.575555,25.040705,動物醫院,308077.995201,2.770410e+06
3222,202401,87456,121.558723,25.033256,長照服務,306382.884791,2.769577e+06


In [4]:
print(df['CATEGORY'].unique())

['居家護理' '長照服務' '動物醫院' '地區醫院' '康復之家' '區域醫院' '物理治療所' '特約醫事放射機構']


In [5]:
hos1 = df[df['CATEGORY'].isin(['地區醫院', '區域醫院'])]
hos2 = df[df['CATEGORY'].isin(['居家護理', '長照服務', '動物醫院', '康復之家', '物理治療所', '特約醫事放射機構'])]
hos1

,POI_YM,POI_ID,POI_LON,POI_LAT,CATEGORY,POI_X,POI_Y
14,201901,46025,121.447058,24.989439,地區醫院,295130.185515,2.764682e+06
15,201901,46038,121.476358,25.005621,地區醫院,298081.744954,2.766485e+06
25,201912,45684,121.498502,25.069025,地區醫院,300291.050816,2.773516e+06
26,201912,45760,121.508539,25.050179,地區醫院,301311.491928,2.771432e+06
42,202001,46412,121.490786,25.060969,區域醫院,299515.853002,2.772620e+06
...,...,...,...,...,...,...,...
3170,202401,48813,121.448888,24.991182,地區醫院,295314.286136,2.764876e+06
3175,202401,48833,121.455012,24.982293,地區醫院,295935.802115,2.763893e+06
3177,202401,48598,121.469648,25.046523,地區醫院,297388.743435,2.771013e+06
3199,202401,48652,121.498691,25.068937,地區醫院,300310.154217,2.773506e+06


In [6]:
p_taipei = pd.read_csv("points_taipei.csv")
p_taipei['index'] = range(0, len(p_taipei))
p_taipei

,x1,y1,x2,y2,cx,cy,index
0,296103,2777535,296203,2777635,296153,2777585,0
1,296103,2777635,296203,2777735,296153,2777685,1
2,296103,2777735,296203,2777835,296153,2777785,2
3,296103,2777835,296203,2777935,296153,2777885,3
4,296103,2777935,296203,2778035,296153,2777985,4
...,...,...,...,...,...,...,...
27656,317103,2768935,317203,2769035,317153,2768985,27656
27657,317103,2769035,317203,2769135,317153,2769085,27657
27658,317103,2769135,317203,2769235,317153,2769185,27658
27659,317103,2769235,317203,2769335,317153,2769285,27659


In [7]:
p_new_taipei = pd.read_csv("points_new_taipei.csv")
p_new_taipei['index'] = range(0, len(p_new_taipei))
p_new_taipei

,x1,y1,x2,y2,cx,cy,index
0,278508,2777870,278608,2777970,278558,2777920,0
1,278508,2777970,278608,2778070,278558,2778020,1
2,278508,2778070,278608,2778170,278558,2778120,2
3,278508,2778170,278608,2778270,278558,2778220,3
4,278508,2778370,278608,2778470,278558,2778420,4
...,...,...,...,...,...,...,...
209743,351508,2767770,351608,2767870,351558,2767820,209743
209744,351608,2766970,351708,2767070,351658,2767020,209744
209745,351608,2767070,351708,2767170,351658,2767120,209745
209746,351608,2767270,351708,2767370,351658,2767320,209746


In [8]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon

In [9]:
# 創建一個包含醫院位置的GeoDataFrame
hospital_points1 = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(hos1['POI_X'], hos1['POI_Y'])])

hos1_100 = hospital_points1.buffer(100)
hos1_500 = hospital_points1.buffer(500)
hos1_1000 = hospital_points1.buffer(1000)
hos1_2000 = hospital_points1.buffer(2000)

hos1_100_gdf = gpd.GeoDataFrame(geometry=hos1_100, crs=hospital_points1.crs)
hos1_500_gdf = gpd.GeoDataFrame(geometry=hos1_500, crs=hospital_points1.crs)
hos1_1000_gdf = gpd.GeoDataFrame(geometry=hos1_1000, crs=hospital_points1.crs)
hos1_2000_gdf = gpd.GeoDataFrame(geometry=hos1_2000, crs=hospital_points1.crs)

In [10]:
# 創建一個包含醫院位置的GeoDataFrame2
hospital_points2 = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(hos2['POI_X'], hos2['POI_Y'])])

hos2_100 = hospital_points2.buffer(100)
hos2_1000 = hospital_points2.buffer(1000)
hos2_2000 = hospital_points2.buffer(2000)

hos2_100_gdf = gpd.GeoDataFrame(geometry=hos2_100, crs=hospital_points2.crs)
hos2_1000_gdf = gpd.GeoDataFrame(geometry=hos2_1000, crs=hospital_points2.crs)
hos2_2000_gdf = gpd.GeoDataFrame(geometry=hos2_2000, crs=hospital_points2.crs)

In [11]:
# 將百米方格加載為GeoDataFrame

def create_geometry_gdf(df):
    geometry = [Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]) for x1, x2, y1, y2 in zip(df['x1'], df['x2'], df['y1'], df['y2'])]
    geometry_gdf = gpd.GeoDataFrame(geometry=geometry)
    geometry_gdf['index'] = range(0, len(df))
    return geometry_gdf

geometry_gdf_t = create_geometry_gdf(p_taipei)
geometry_gdf_nt = create_geometry_gdf(p_new_taipei)
geometry_gdf_t


,geometry,index
0,"POLYGON ((296103.000 2777535.000, 296203.000 2...",0
1,"POLYGON ((296103.000 2777635.000, 296203.000 2...",1
2,"POLYGON ((296103.000 2777735.000, 296203.000 2...",2
3,"POLYGON ((296103.000 2777835.000, 296203.000 2...",3
4,"POLYGON ((296103.000 2777935.000, 296203.000 2...",4
...,...,...
27656,"POLYGON ((317103.000 2768935.000, 317203.000 2...",27656
27657,"POLYGON ((317103.000 2769035.000, 317203.000 2...",27657
27658,"POLYGON ((317103.000 2769135.000, 317203.000 2...",27658
27659,"POLYGON ((317103.000 2769235.000, 317203.000 2...",27659


In [12]:
def overlay_and_count(geometry_gdf, hospital_gdf, prefix):
    intersection = geometry_gdf.overlay(hospital_gdf, how='intersection')
    count_df = intersection.groupby('index').size().reset_index(name=prefix + '_cnt')
    return count_df

def calculate_hospital_counts(geometry_gdf, hos1_100_gdf, hos1_500_gdf, hos1_1000_gdf, hos1_2000_gdf, hos2_100_gdf, hos2_1000_gdf, hos2_2000_gdf):
    hos1_100_cnt = overlay_and_count(geometry_gdf, hos1_100_gdf, 'hos1_100')
    hos1_500_cnt = overlay_and_count(geometry_gdf, hos1_500_gdf, 'hos1_500')
    hos1_1000_cnt = overlay_and_count(geometry_gdf, hos1_1000_gdf, 'hos1_1000')
    hos1_2000_cnt = overlay_and_count(geometry_gdf, hos1_2000_gdf, 'hos1_2000')
    hos2_100_cnt = overlay_and_count(geometry_gdf, hos2_100_gdf, 'hos2_100')
    hos2_1000_cnt = overlay_and_count(geometry_gdf, hos2_1000_gdf, 'hos2_1000')
    hos2_2000_cnt = overlay_and_count(geometry_gdf, hos2_2000_gdf, 'hos2_2000')
    return hos1_100_cnt, hos1_500_cnt, hos1_1000_cnt, hos1_2000_cnt, hos2_100_cnt, hos2_1000_cnt, hos2_2000_cnt

# 呼叫函式
hos1_100_cnt_t, hos1_500_cnt_t, hos1_1000_cnt_t, hos1_2000_cnt_t, hos2_100_cnt_t, hos2_1000_cnt_t, hos2_2000_cnt_t = calculate_hospital_counts(geometry_gdf_t, hos1_100_gdf, hos1_500_gdf, hos1_1000_gdf, hos1_2000_gdf, hos2_100_gdf, hos2_1000_gdf, hos2_2000_gdf)
hos1_100_cnt_nt, hos1_500_cnt_nt, hos1_1000_cnt_nt, hos1_2000_cnt_nt, hos2_100_cnt_nt, hos2_1000_cnt_nt, hos2_2000_cnt_nt = calculate_hospital_counts(geometry_gdf_nt, hos1_100_gdf, hos1_500_gdf, hos1_1000_gdf, hos1_2000_gdf, hos2_100_gdf, hos2_1000_gdf, hos2_2000_gdf)


In [14]:
def calculate_weighted_score(df, *args):
    # 進行多個 merge 操作
    for arg in args:
        df = df.merge(arg, on='index', how='left')
    
    # 將 NaN 值填充為 0
    df = df.fillna(0)
    
    # 計算加權分數
    df['醫院加權分數'] = (
        df['hos1_100_cnt'] * 1 +
        df['hos1_500_cnt'] * 0.7 +
        df['hos1_1000_cnt'] * 0.5 +
        df['hos1_2000_cnt'] * 0.2 +
        df['hos2_100_cnt'] * 0.7 +
        df['hos2_1000_cnt'] * 0.4 +
        df['hos2_2000_cnt'] * 0.2
    )
    
    return df

# 呼叫函式
result_t = calculate_weighted_score(p_taipei, hos1_100_cnt_t, hos1_500_cnt_t, hos1_1000_cnt_t, hos1_2000_cnt_t, hos2_100_cnt_t, hos2_1000_cnt_t, hos2_2000_cnt_t)
result_nt = calculate_weighted_score(p_new_taipei, hos1_100_cnt_nt, hos1_500_cnt_nt, hos1_1000_cnt_nt, hos1_2000_cnt_nt, hos2_100_cnt_nt, hos2_1000_cnt_nt, hos2_2000_cnt_nt)

In [15]:
result_t.sort_values('醫院加權分數', ascending=False)

,x1,y1,x2,y2,cx,cy,index,hos1_100_cnt,hos1_500_cnt,hos1_1000_cnt,hos1_2000_cnt,hos2_100_cnt,hos2_1000_cnt,hos2_2000_cnt,醫院加權分數
12460,305503,2770335,305603,2770435,305553,2770385,12460,0.0,5.0,8.0,13.0,7.0,84.0,259.0,100.4
11268,305003,2770135,305103,2770235,305053,2770185,11268,3.0,6.0,7.0,10.0,5.0,75.0,271.0,100.4
12223,305403,2770335,305503,2770435,305453,2770385,12223,0.0,6.0,8.0,12.0,7.0,83.0,258.0,100.3
12225,305403,2770535,305503,2770635,305453,2770585,12225,0.0,3.0,8.0,12.0,6.0,85.0,265.0,99.7
11504,305103,2770035,305203,2770135,305153,2770085,11504,3.0,6.0,6.0,10.0,5.0,75.0,266.0,98.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14417,306203,2788235,306303,2788335,306253,2788285,14417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14416,306203,2788135,306303,2788235,306253,2788185,14416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14415,306203,2788035,306303,2788135,306253,2788085,14415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14414,306203,2787935,306303,2788035,306253,2787985,14414,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
result_nt.sort_values('醫院加權分數', ascending=False)

,x1,y1,x2,y2,cx,cy,index,hos1_100_cnt,hos1_500_cnt,hos1_1000_cnt,hos1_2000_cnt,hos2_100_cnt,hos2_1000_cnt,hos2_2000_cnt,醫院加權分數
79206,302008,2766870,302108,2766970,302058,2766920,79206,0.0,3.0,5.0,9.0,6.0,86.0,210.0,87.0
47989,296508,2765970,296608,2766070,296558,2766020,47989,3.0,3.0,4.0,15.0,5.0,79.0,207.0,86.6
79709,302108,2766870,302208,2766970,302158,2766920,79709,0.0,3.0,5.0,9.0,7.0,87.0,202.0,86.5
79710,302108,2766970,302208,2767070,302158,2767020,79710,1.0,3.0,5.0,10.0,7.0,82.0,204.0,86.1
79708,302108,2766770,302208,2766870,302158,2766820,79708,1.0,3.0,5.0,9.0,6.0,88.0,195.0,85.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103839,307708,2742170,307808,2742270,307758,2742220,103839,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103838,307708,2742070,307808,2742170,307758,2742120,103838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103837,307708,2741970,307808,2742070,307758,2742020,103837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103836,307708,2741870,307808,2741970,307758,2741920,103836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# 寫入csv
result_t.to_csv("taipei_hos.csv", index=False)
result_nt.to_csv("new_taipei_hos.csv", index=False)